<a href="https://colab.research.google.com/github/asrinivasan75/CariesClassification/blob/main/DentalImageCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

base_dir = '/content/drive/MyDrive/Dental_Model'
nocaries_dir = os.path.join(base_dir, 'NoCaries')
interproximalcaries_dir = os.path.join(base_dir, 'InterproximalCaries')


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,          #randomization
    zoom_range=0.2,           #randomization
    horizontal_flip=True,
    validation_split=0.4      #splitting data
)

#generators for training/testing
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',      # classifying as one or the other
    subset='training'         # set as training data
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'       # set as testing data
)

Found 17 images belonging to 2 classes.
Found 11 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  # trying to prevent overfitting (practically impossible with size of data)
model.add(Dense(1, activation='sigmoid'))  # sigmoid --> binary output layer

# creating the model --> adam is preferred optimizer
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.5294 - loss: 0.6852 - val_accuracy: 0.6364 - val_loss: 1.9617
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.6471 - loss: 2.1621 - val_accuracy: 0.3636 - val_loss: 0.7924
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.3529 - loss: 0.8547 - val_accuracy: 0.3636 - val_loss: 0.7938
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.3529 - loss: 0.8426 - val_accuracy: 0.6364 - val_loss: 0.6708
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.6471 - loss: 0.6417 - val_accuracy: 0.6364 - val_loss: 0.6485
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.6471 - loss: 0.6502 - val_accuracy: 0.6364 - val_loss: 0.6549
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.6471 - loss: 0.6536 - val_accuracy: 0.6364 - val_loss: 0.6508
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.6471 - loss: 0.6414 - val_accuracy: 0.6364 - val_loss: 0.6505
Epoch 9/10
1/1 ━

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 506ms/step - accuracy: 0.6364 - loss: 0.6561
Validation Accuracy: 63.64%
